In [1]:
!pip install tensorflow
!pip3 install tensorflow_text>=2.0.0rc0
! pip install pymorphy2

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.python.keras import backend as K
import tensorflow_text
import numpy as np
import pymorphy2

In [5]:
print(tf.__version__)

2.2.0


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
urls_to_id = dict() # url -> id
id_to_urls=  dict() #id -> url
i= 0
with open("/content/drive/My Drive/Colab Notebooks/urls.numerate.txt", encoding='utf-8') as f:
    i = 0
    for s in f:
        tt = s.split('\t')
        id_ = int(tt[0])
        urls_to_id[tt[1][:-1]] = id_
        id_to_urls[id_] = tt[1][:-1]
        i += 1
print(i, len(urls_to_id), len(id_to_urls))

38114 38114 38114


In [11]:
queries = []
i= 0
with open("/content/drive/My Drive/Colab Notebooks/queries.corr.txt", encoding='utf-8') as f:
    i = 0
    for s in f:
        tt = s.split('\t')
        queries.append(tt[1][:-1])
        i += 1
print(i, len(queries))

399 399


In [0]:
quer_to_id = dict() #quer_num -> list_id
with open("/content/drive/My Drive/Colab Notebooks/sample.submission.txt", encoding='utf-8') as f:
    temp = f.readline()
    for line in f:
        q, ii = line.split(',')
        q = int(q)
        ii = int(ii[:-1])
        if q not in quer_to_id:
            quer_to_id[q] = []
        quer_to_id[q].append(ii)

In [0]:
url_to_title = dict()
titles  =[]
with open("/content/drive/My Drive/Colab Notebooks/titles_norm.txt", "r", encoding='cp1251',errors = 'ignore') as f:
    for s in f:
      u, t = s.split('\t')
      t = t[:-1]
      titles.append(t)
      url_to_title[u] = t

In [2]:
answer = []
stop_words = ['как', "где", "что", "почему", "ли", "какой"]
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

# for quer_num in range(1):
for quer_num in range(len(queries)):
    if quer_num == 394 or quer_num == 398:
        continue
    print(quer_num)
    ids = quer_to_id[quer_num +1]
    texts_title = []
    texts_all = []
    for id_ in ids:
        try:
            texts_title.append(url_to_title[id_to_urls[id_]])
        except:
            print(id_, '####')
            texts_title.append('')

    with open("/content/drive/My Drive/Colab Notebooks/text_quer/" + str(quer_num + 1), encoding='cp1251') as f:
        for i in f:
            texts_all.append(i[:5500])

    assert len(texts_title) == len(texts_all) 

    morph = pymorphy2.MorphAnalyzer()
    tokenized_query = [morph.parse(i)[0].normal_form \
                       for i in queries[quer_num].split() \
                      if i not in stop_words]
                      
    quer = ' '.join(tokenized_query)


    question_embeddings = module.signatures['question_encoder'](
            tf.constant(quer))
    response_embeddings = module.signatures['response_encoder'](
            input=tf.constant(texts_title),
            context=tf.constant(texts_all))

    dists = np.inner(question_embeddings['outputs'], response_embeddings['outputs'])
    top_rangs = list(np.argsort(dists[0]))[::-1]
    top_rangs = top_rangs[:10]
    
    for i in top_rangs:
        answer.append((quer_num + 1, ids[i]))
    

In [20]:
answer[:30]

[(1, 74),
 (1, 28),
 (1, 53),
 (1, 78),
 (1, 44),
 (1, 7),
 (1, 66),
 (1, 57),
 (1, 58),
 (1, 17),
 (2, 154),
 (2, 156),
 (2, 165),
 (2, 125),
 (2, 185),
 (2, 103),
 (2, 192),
 (2, 131),
 (2, 141),
 (2, 179),
 (3, 228),
 (3, 201),
 (3, 242),
 (3, 271),
 (3, 217),
 (3, 198),
 (3, 275),
 (3, 294),
 (3, 293),
 (3, 225)]

In [0]:
with open("/content/drive/My Drive/Colab Notebooks/predict2.csv", "w") as f:
    f.write("QueryId,DocumentId\n")
    for i in answer:
        f.write("{},{}\n".format(i[0], i[1]))